In [1]:
def create_small_input_array(list_of_img_path, args):
    """
    list_of_img_path: list of training image and corresponding target image path
		e.g. ["train_im_1_OR.png","train_im_2_OR.png",]
	"""
    train_list, target_list, hist_list = list(), list(), list()
    for img_path in list_of_img_path:
        tmp_X, tmp_y, tmp_hist_list = load_one_data(img_path,
                                                    img_path.replace("OR", "GT"), args)
        if args.add_feature != None:
            feature = scipy.misc.imread(img_path.replace("OR", args.add_feature),
                flatten=False, mode='L').reshape(args.H,args.W,1).astype('float32')/255
            tmp_X = np.dstack((tmp_X, feature))
            tmp_X = np.array(tmp_X)
        train_list.append(tmp_X);target_list.append(tmp_y)
        if tmp_hist_list != None:
            hist_list.append(tmp_hist_list)
    return train_list, target_list, hist_list

def crnn_gen_shuffle(x_set, batch_size, args, n):
    if n == 2:
        case_list = x_set.copy()
        np.random.shuffle(case_list)
        x_set = create_crnn_gen_file_list(case_list, args.training_path, args,n=n)
        steps = int(np.floor(len(x_set) / float(batch_size)))
        while True:
            for i in range(steps):
                batch_x = x_set[i * batch_size:(i + 1) * batch_size]
                batch_X_0, batch_X_1, batch_Y, batch_Y_right = [], [], [], []
                # print(batch_x)
                for idx in range(len(batch_x)):
                    tmp_x, tmp_y, tmp_hist_list = create_small_input_array(batch_x[idx], args)
                    tmp_x = kw_preprocessing(np.array(tmp_x), args)
                    if not args.reverse:
                        batch_X_0.append(np.array(tmp_x[0]))
                        batch_X_1.append(np.array(tmp_x[1]))
                        batch_Y.append(np.array(tmp_y[1]))
                    else:
                        batch_X_0.append(np.array(tmp_x[1]))
                        batch_X_1.append(np.array(tmp_x[0]))
                        batch_Y.create_small_input_array(np.array(tmp_y[0]))
                # if np.sum(batch_Y)==0:
                #     yield [np.array(batch_X_0), np.array(batch_X_1)], np.array(batch_Y)
                yield [np.array(batch_X_0), np.array(batch_X_1)], np.array(batch_Y)

    elif n == 3:
        case_list = x_set.copy()
        np.random.shuffle(case_list)
        x_set = create_crnn_gen_file_list(case_list, args.training_path, args,n=n)
        steps = int(np.floor(len(x_set) / float(batch_size)))

        def create_small_input3_array(list_of_img_path, args):
            train_list, target_list = list(), list()
            for img_path in list_of_img_path:
                tmp_X, tmp_y, _ = load_one_data(img_path, img_path.replace("OR", "GT"), args)
                
                if args.add_feature != None:
                    feature = scipy.misc.imread(img_path.replace("OR", args.add_feature),
                        flatten=False, mode='L').reshape(args.H, args.W,1).astype('float32')/255
                    tmp_X = np.dstack((tmp_X, feature))
                    tmp_X = np.array(tmp_X)

                train_list.append(tmp_X);target_list.append(tmp_y)           
            return train_list, target_list

        while True:
            using_batch = []
            for i in range(steps):
                using_batch.append(x_set[i * batch_size:(i + 1) * batch_size])
            np.random.shuffle(using_batch)

            for batch_x in using_batch:

                batch_X_0, batch_X_1, batch_X_2, batch_Y = [], [], [], []
                for idx in range(len(batch_x)):
                    tmp_x, tmp_y = create_small_input3_array(batch_x[idx], args)
                    batch_X_0.append(np.array(tmp_x[0]))
                    batch_X_1.append(np.array(tmp_x[1]))
                    batch_X_2.append(np.array(tmp_x[2]))
                    batch_Y.append(np.array(tmp_y[1]))
                    
                # if np.sum(batch_Y)==0:
                #     yield [np.array(batch_X_0), np.array(batch_X_1), np.array(batch_X_2)], np.array(batch_Y)
                #     yield [np.array(batch_X_0), np.array(batch_X_1), np.array(batch_X_2)], np.array(batch_Y)                                        
                yield [np.array(batch_X_0), np.array(batch_X_1), np.array(batch_X_2)], np.array(batch_Y)
            
    else:
        print("still support n = 2 or 3")

def create_crnn_gen_file_list(case_list, case_path_root, args,  n=2):
    out_x = []
    for case in case_list:
        img_list = create_img_path_list(case_path_root + case)
        out_x += [img_list[i:i + n] for i in range(0, len(img_list) - n + 1)]
    return out_x

def crnn_gen(x_set, batch_size, args, n):
    steps = int(np.floor(len(x_set) / float(batch_size)))
    if n == 2:
        while True:
            for i in range(steps):
                batch_x = x_set[i * batch_size:(i + 1) * batch_size]
                batch_X_0, batch_X_1, batch_Y = [], [], []
                pair = []
                for idx in range(len(batch_x)):
                    tmp_x, tmp_y, tmp_hist_list = create_small_input_array(batch_x[idx], args)
                    tmp_x = kw_preprocessing(np.array(tmp_x), args)
                    # yield [np.array([tmp_x[0]]), np.array([tmp_x[1]])] , np.array([tmp_y[0]])#[np.array([tmp_y[0]]), np.array([tmp_y[1]])]
                    batch_X_0.append(np.array(tmp_x[0]))
                    batch_X_1.append(np.array(tmp_x[1]))
                    # pair.append(np.array[tmp_x])
                    batch_Y.append(np.array(tmp_y[1]))
                yield [np.array(batch_X_0), np.array(batch_X_1)], np.array(batch_Y)
    elif n == 3:
        while True:
            for i in range(steps):
                batch_x = x_set[i * batch_size:(i + 1) * batch_size]
                for idx in range(len(batch_x)):
                    tmp_x, tmp_y, tmp_hist_list = create_small_input_array(batch_x[idx], args)
                    tmp_x = kw_preprocessing(np.array(tmp_x), args)
                    yield [np.array([tmp_x[0]]), np.array([tmp_x[1]]), np.array([tmp_x[2]])], np.array([tmp_y[1]])
    else:
        print("still support n = 2 or 3")


def large_dataset_gen(x_set, batch_size, args):
    steps = int(np.floor(len(x_set) / float(batch_size)))
    while True:
        for i in range(steps):
            batch_x = x_set[i * batch_size:(i + 1) * batch_size]
            X, Y = [], []
            for idx in range(len(batch_x)):
                tmp_x, tmp_y, tmp_hist_list = create_small_input_array([batch_x[idx]], args)
                tmp_x = kw_preprocessing(np.array(tmp_x), args)
                X.append(tmp_x[0]);
                Y.append(np.array(tmp_y[0]))
            yield np.array(X), np.array(Y)
        np.random.shuffle(x_set)


def create_caselist(args):
    df = pd.read_csv(args.index)
    # case_list = os.listdir(args.training_path)
    # num_case = len(case_list)
    # split_num = [int(np.round(num_case*.6)),int(np.round(num_case*.8))] # train:valid:test = 6:2:2
    train = df['train'].dropna().tolist()
    valid = df['validation'].dropna().tolist()
    test  = df['test'].dropna().tolist()

### 從學長程式碼找到的對應loss

In [ ]:
def weightedConsecutiveLoss(args):
    def DL(y_true,y_pred):
	    return 1-((K.sum(y_true*y_pred,axis=[1,2])+K.epsilon())/(K.sum(y_true+y_pred,axis=[1,2])+K.epsilon()))-((K.sum((1-y_true)*(1-y_pred),axis=[1,2])+K.epsilon())/(K.sum((2-y_true-y_pred),axis=[1,2])+K.epsilon()))
    def weightedConsecutiveDiceLoss(y_true, y_pred):
        if args.loss_ratio != None and args.loss_ratio != 0:
            if args.consecutive == 2:
                currentRatio = ( args.loss_ratio / ( args.loss_ratio + 1 ) )
                loss_1 = currentRatio * K.mean( DL( y_true, y_pred), 0 )
                anotherRatio = ( 1 / (args.loss_ratio + 1 ) )
                loss_2 = anotherRatio * K.mean( DL( y_true[:-1], y_pred[1:] ), 0 )
                print("Using ratio [prev: curr]=[{:.2f},{:.2f}]".format(anotherRatio, currentRatio) )
                return K.mean(tf.concat([loss_1,loss_2], 0), 0)
            else: # args.consecutive == 3
                currentRatio = ( args.loss_ratio * 2 / ( args.loss_ratio * 2 + 2 ) )
                lossCenter = currentRatio * K.mean( DL( y_true, y_pred), 0 )
                anotherRatio = ( 1 / ( args.loss_ratio * 2 + 2 ) )
                lossLeft  = anotherRatio * K.mean( DL( y_true[:-1], y_pred[1:] ), 0 )
                lossRight = anotherRatio * K.mean( DL( y_true[1:], y_pred[:-1] ), 0 )
                print("Using ratio [prev: curr: next]=[{:.2f},{:.2f},{:.2f}]".format(anotherRatio,
                                                            currentRatio, anotherRatio) )
                res = K.mean(tf.concat([lossLeft, lossCenter, lossRight], 0), 0)
                return res
        elif args.loss_ratio == 0:
            # dynamic weight
            if args.consecutive == 3:
                print("dynamic weight")
                Y = y_true
                Y = K.concatenate([K.expand_dims(Y[0],0), Y, K.expand_dims(Y[-1],0)], axis=0)
                prevCurrDiff = K.sum(K.abs(Y[1:]-Y[:-1]), axis=(1,2,3))
                prevCurrDiffDivideSum = prevCurrDiff / K.sum(Y[:-1], axis=(1,2,3))*120
                prevCurrDiffDivideSum = K.tf.where(K.tf.is_nan(prevCurrDiffDivideSum), K.zeros_like(prevCurrDiffDivideSum), prevCurrDiffDivideSum)
                const = 2
                mask = K.equal(prevCurrDiffDivideSum, K.zeros_like(prevCurrDiffDivideSum)) 
                mask = K.cast(mask, dtype='float32')
                centerWeight = (const - 1) * mask + prevCurrDiffDivideSum
                batchWeight = K.concatenate([K.expand_dims(centerWeight[1:]),
                            K.expand_dims(K.tf.multiply(centerWeight[1:], centerWeight[:-1])),
                            K.expand_dims(centerWeight[:-1])], axis=1)
                denominator = K.sum(batchWeight, axis=1)


                currentRatio = K.expand_dims(K.tf.multiply(centerWeight[1:], centerWeight[:-1])/ denominator)
                currentRatio = K.tf.where(K.tf.is_nan(currentRatio), K.ones_like(currentRatio)/3, currentRatio)
                prevRatio = K.expand_dims(centerWeight[1:] / denominator)
                prevRatio = K.tf.where(K.tf.is_nan(prevRatio), K.ones_like(prevRatio)/3, prevRatio)
                nextRatio = K.expand_dims(centerWeight[:-1] / denominator)
                nextRatio = K.tf.where(K.tf.is_nan(nextRatio), K.ones_like(nextRatio)/3, nextRatio)

                lossCenter = K.mean(currentRatio * DL( Y[1:-1], y_pred), 0 )
                lossLeft  = K.mean(prevRatio * DL( Y[:-2], y_pred ), 0 )
                lossRight = K.mean(nextRatio * DL( Y[2:], y_pred ), 0 )


                res = K.mean(tf.concat([lossLeft, lossCenter, lossRight], 0), 0)
                return res#K.mean(tf.concat([lossLeft, lossCenter, lossRight], 0), 0)
            else:
                print("not for 2")
                # sys.exit()
    weightedConsecutiveDiceLoss.__name__ = "weightedConsecutiveDiceLoss"
    return weightedConsecutiveDiceLoss

### 新的model (待verify)

#### dataloader

In [ ]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset
from pathlib import Path

class CRNNDataset(Dataset):
    def __init__(self, root_dir, target_dir=None, n_frames=3, is_train=True, 
                 use_blur=False, use_grad=False, use_hist=False):
        """
        Args:
            root_dir: 訓練圖片的根目錄 (裡面包含各個 Case 的資料夾，或直接是圖片)
            target_dir: Ground Truth 的根目錄
            n_frames: 2 或 3 (對應學長的 consecutive)
            use_blur: 是否使用高斯模糊特徵 (學長的 args.blur)
            use_grad: 是否使用梯度特徵 (學長的 args.gradient_preprocess)
            use_hist: 是否使用直方圖特徵 (學長的 input_hist)
        """
        self.root_dir = root_dir
        self.target_dir = target_dir
        self.n_frames = n_frames
        self.is_train = is_train
        
        # 特徵工程開關
        self.use_blur = use_blur
        self.use_grad = use_grad
        self.use_hist = use_hist

        # 1. 建立滑動視窗索引 (Sliding Window Indexing)
        # 格式: [ (prev_path, curr_path, next_path), label_path ]
        self.samples = self._make_dataset()

    def _make_dataset(self):
        """
        模擬學長的 create_crnn_gen_file_list
        """
        samples = []
        
        # 假設 root_dir 裡面是各個 Case 的資料夾 (例如: case1, case2...)
        # 如果你的結構是直接一堆 png，請把 root_dir 當作單一 case 處理
        if not os.path.exists(self.root_dir):
            raise ValueError(f"Directory not found: {self.root_dir}")

        # 這裡假設 root_dir 下面有很多子資料夾，每個資料夾代表一個 Volume
        # 如果你的資料結構是扁平的 (直接在 root_dir 下就是 1.png, 2.png...)
        # 那就把 case_dirs 設為 [self.root_dir]
        case_dirs = [os.path.join(self.root_dir, d) for d in os.listdir(self.root_dir) 
                     if os.path.isdir(os.path.join(self.root_dir, d))]
        """
        以上的寫法等價於
        case_dirs = []
        for d in os.listdir(self.root_dir):
        full_path = os.path.join(self.root_dir, d)
        if os.path.isdir(full_path):
        case_dirs.append(full_path)
        """
        # 如果找不到子資料夾，就假設 root_dir 本身就是一個 case
        ## 視為單一case 
        if not case_dirs:
            case_dirs = [self.root_dir]

        for case_path in case_dirs:
            # 讀取該 Case 下所有 png 並排序 (學長的 create_img_path_list 邏輯)
            # 排序很重要！必須是 1.png, 2.png, 3.png...
            imgs = sorted([f for f in os.listdir(case_path) if f.endswith('.png')],
                          key=lambda x: int(os.path.splitext(x)[0]))
            
            full_img_paths = [os.path.join(case_path, f) for f in imgs]
            ## 此時full_img_paths為儲存此迴圈中的所有path之排序
            
            if self.target_dir:
                # 假設 Target 的結構跟 Input 一模一樣
                case_name = os.path.basename(case_path)
                ## case_path是在root_dir後 png檔案外面那層資料夾
                target_case_path = os.path.join(self.target_dir, case_name)
                # 如果 target 結構是扁平的，要自己調整這裡
                if not os.path.exists(target_case_path): 
                     target_case_path = self.target_dir # Fallback
                
                full_target_paths = [os.path.join(target_case_path, f) for f in imgs]
            
            # 製作滑動視窗
            # 如果 n=3, 會有 [i, i+1, i+2]
            for i in range(len(full_img_paths) - self.n_frames + 1):
                window_imgs = full_img_paths[i : i + self.n_frames]
                
                # Label 對應的是中間那張 (curr) 或是最後一張?
                # 學長邏輯: return batch_Y.append(tmp_y[1]) -> n=3 時取中間
                # n=2 時取後面那個? 學長代碼: yield ... np.array(batch_Y) (batch_Y append tmp_y[1])
                # 結論：Label 總是取 window 中的 index=1 (第二張)
                
                label_path = None
                if self.target_dir:
                    # 取 window 中間 (index 1) 的 label
                    # 如果 n=2, index 1 就是第二張 (Curr)
                    label_idx = i + 1 
                    label_path = full_target_paths[label_idx]
                
                samples.append((window_imgs, label_path))
                
        return samples ## return的samples 會是一個系列的切片位置 對應這組系列切片應該產出的GT位置

    def _load_image(self, path): ## 讀image並且做標準化
        # 替換 scipy.misc.imread -> cv2.imread
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError(f"Failed to load image: {path}")
        img = img.astype(np.float32)
        
        # Normalization (0~1)
        # 學長邏輯: (img - min) / (max - min)
        img_min, img_max = img.min(), img.max()
        if img_max > img_min:
            img = (img - img_min) / (img_max - img_min)
        else:
            img = img * 0 # 避免除以 0
            
        return img

    def _preprocess(self, img_np):
        """
        重現學長的 kw_preprocessing (Blur, Gradient)
        img_np shape: (H, W)
        Return shape: (C, H, W) -> C 可能是 1, 3, 5... 取決於特徵
        """
        processed_channels = [img_np] # 原始圖
        
        # 1. Blur (高斯模糊)
        if self.use_blur:
            # 學長用的是 sigma=[2], kernel=7
            blur = cv2.GaussianBlur(img_np, (7, 7), 2)
            # Normalize blur
            b_min, b_max = blur.min(), blur.max()
            if b_max > b_min:
                blur = (blur - b_min) / (b_max - b_min)
            processed_channels.append(blur)
            
        # Stack 起來變成 (H, W, C_temp) 為了做 Gradient
        feature_stack = np.stack(processed_channels, axis=-1) 
        
        # 2. Gradient (Sobel + Laplacian)
        if self.use_grad:
            grad_channels = []
            ksize = 5
            # 對每個現有的 channel (Original, Blur) 做 gradient
            for i in range(feature_stack.shape[-1]):
                src = feature_stack[:, :, i]
                
                # Laplacian
                lap = cv2.Laplacian(src, cv2.CV_32F, ksize=ksize)
                # Normalize Lap
                l_min, l_max = lap.min(), lap.max()
                if l_max > l_min: lap = (lap - l_min) / (l_max - l_min)
                
                # Sobel
                sobelx = cv2.Sobel(src, cv2.CV_32F, 1, 0, ksize=ksize)
                sobely = cv2.Sobel(src, cv2.CV_32F, 0, 1, ksize=ksize)
                sobel = np.maximum(np.abs(sobelx), np.abs(sobely))
                # Normalize Sobel
                s_min, s_max = sobel.min(), sobel.max()
                if s_max > s_min: sobel = (sobel - s_min) / (s_max - s_min)
                
                grad_channels.append(sobel)
                grad_channels.append(lap)
            
            # 將 Gradient 特徵加回去
            # 學長邏輯：[Original, Blur, Sobel_Org, Lap_Org, Sobel_Blur, Lap_Blur]
            # 這裡我們簡化處理，全部疊加
            processed_channels.extend(grad_channels)

        # Final Stack: (H, W, C) -> Transpose to (C, H, W) for PyTorch
        final_img = np.stack(processed_channels, axis=0) 
        return torch.from_numpy(final_img).float()

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        window_paths, label_path = self.samples[idx]
        
        # 讀取並處理 Input (Prev, Curr, Next)
        tensors = []
        for p in window_paths:
            img = self._load_image(p)    # Load & Norm
            tensor = self._preprocess(img) # Blur & Grad & ToTensor
            tensors.append(tensor) ## tensor w/ the shape of (C,H,W)
            
        # 根據 n_frames 組裝 dict
        sample = {}
        if self.n_frames == 3:
            sample = {
                'prev': tensors[0],
                'curr': tensors[1],
                'next': tensors[2]
            }
        elif self.n_frames == 2:
            sample = {
                'prev': tensors[0],
                'curr': tensors[1]
            }

        # 讀取 Label
        if self.target_dir and label_path:
            # Label 不需要做 Blur/Grad，只需要讀取跟 Normalize (通常是 0/1)
            lbl = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
            lbl = (lbl / 255.0).astype(np.float32) # 轉成 0~1
            lbl = torch.from_numpy(lbl).unsqueeze(0) # (1, H, W)
            sample['label'] = lbl
            
        return sample
    



#### loss function

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DoubleDiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DoubleDiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, y_true, y_pred):
        """
        PyTorch Shape: [Batch, Channel, H, W]
        TF Original: axis=[1,2] -> H, W (Channel is last in TF)
        PyTorch Equivalent: Sum over [2, 3] if input is [B, C, H, W]
        """
        # 確保輸入在 0~1 之間 (如果模型最後沒有 Sigmoid，這裡要加)
        # y_pred = torch.sigmoid(y_pred) 
        
        # 展平 H, W 維度以便計算
        # [B, C, H, W] -> [B, C, H*W]
        batch, channel, h, w = y_true.shape
        y_true_f = y_true.view(batch, channel, -1)
        y_pred_f = y_pred.view(batch, channel, -1)

        # Term 1: Foreground Dice
        intersection = torch.sum(y_true_f * y_pred_f, dim=2)
        union = torch.sum(y_true_f + y_pred_f, dim=2)
        term1 = (intersection + self.smooth) / (union + self.smooth)

        # Term 2: Background Dice (1 - y)
        intersection_inv = torch.sum((1 - y_true_f) * (1 - y_pred_f), dim=2)
        union_inv = torch.sum((2 - y_true_f - y_pred_f), dim=2)
        term2 = (intersection_inv + self.smooth) / (union_inv + self.smooth)

        # Final Loss: 1 - Term1 - Term2
        # 注意：通常 Dice Loss 是 1 - Dice。這裡是 1 - FG_Dice - BG_Dice
        # 但這可能導致負值 (因為 Dice 最大是 1)，學長原公式是這樣寫的，我們先照搬。
        # 原公式：1 - (FG) - (BG)。如果 FG=1, BG=1，Loss = -1。
        # 建議檢查一下是否應該是 1 - (FG + BG)/2 或者是 2 - FG - BG
        # 為了忠實復刻，這裡保持原樣：
        return 1 - term1 - term2

class WeightedConsecutiveLoss(nn.Module):
    def __init__(self, loss_ratio=0.5, consecutive=3, dynamic_weight=False):
        super(WeightedConsecutiveLoss, self).__init__()
        self.loss_ratio = loss_ratio
        self.consecutive = consecutive
        self.dynamic_weight = dynamic_weight
        self.base_loss = DoubleDiceLoss()

    def forward(self, y_true, y_pred):
        # 假設 y_true, y_pred 形狀為 [Batch, C, H, W]
        
        # 1. 固定權重模式 (Fixed Ratio)
        if not self.dynamic_weight and self.loss_ratio != 0:
            if self.consecutive == 2:
                # 權重計算
                current_ratio = self.loss_ratio / (self.loss_ratio + 1)
                neighbor_ratio = 1 / (self.loss_ratio + 1)
                
                print(f"Ratio: Curr={current_ratio:.2f}, Prev={neighbor_ratio:.2f}")

                loss_center = current_ratio * self.base_loss(y_true, y_pred)
                
                # Cross-Temporal Loss: True[t] vs Pred[t+1]
                # 注意：這裡會讓 Batch Size 變少 1
                loss_prev = neighbor_ratio * self.base_loss(y_true[:-1], y_pred[1:])
                
                # 為了將 loss 合併，我們取 mean
                return torch.mean(loss_center) + torch.mean(loss_prev)

            else: # consecutive == 3 (CRNN Default)
                # 權重計算
                total_share = self.loss_ratio * 2 + 2
                current_ratio = (self.loss_ratio * 2) / total_share
                neighbor_ratio = 1 / total_share

                # loss_center: [Batch]
                loss_center = current_ratio * self.base_loss(y_true, y_pred)
                
                # loss_left: True[t] vs Pred[t+1] (Shifted Left)
                loss_left = neighbor_ratio * self.base_loss(y_true[:-1], y_pred[1:])
                
                # loss_right: True[t] vs Pred[t-1] (Shifted Right)
                loss_right = neighbor_ratio * self.base_loss(y_true[1:], y_pred[:-1])

                # 原代碼使用了 tf.concat 然後 mean。
                # 在 PyTorch 直接加總 mean 即可，數學上等價
                final_loss = torch.mean(loss_center) + torch.mean(loss_left) + torch.mean(loss_right)
                return final_loss

        # 2. 動態權重模式 (Dynamic Weight) - 只有 consecutive=3
        elif self.dynamic_weight or self.loss_ratio == 0:
            if self.consecutive != 3:
                raise ValueError("Dynamic weight only supports consecutive=3")
            
            # --- 計算動態權重 (Dynamic Logic) ---
            # TF: Y = K.concatenate([Y[0], Y, Y[-1]]) padding batch
            # 這是為了計算每一幀跟前後幀的差異
            y_pad = torch.cat([y_true[0:1], y_true, y_true[-1:]], dim=0)
            
            # 差異計算: |Y[t+1] - Y[t]|
            # Sum over (C, H, W) -> dims [1, 2, 3]
            diff = torch.abs(y_pad[1:] - y_pad[:-1])
            prev_curr_diff = torch.sum(diff, dim=[1, 2, 3]) # [Batch+1]
            
            # 歸一化差異
            y_sum = torch.sum(y_true, dim=[1, 2, 3]) # [Batch]
            # y_pad[:-1] 對應的是原長度的 padding 版本，這裡簡化邏輯：
            # 學長原代碼：prevCurrDiff / sum(Y[:-1]) * 120
            # 為了對齊長度，我們取 diff 的前 N 個
            diff_metric = (prev_curr_diff[:-1] / (y_sum + 1e-6)) * 120
            
            # 處理 NaN
            diff_metric = torch.nan_to_num(diff_metric, 0.0)
            
            const = 2.0
            # mask: if diff_metric == 0
            mask = (diff_metric == 0).float()
            center_weight_raw = (const - 1) * mask + diff_metric
            
            # 建構三項權重
            # Batch Weight: [Left, Center, Right]
            # Center = diff[t] * diff[t-1]
            cw_curr = center_weight_raw[1:]
            cw_prev = center_weight_raw[:-1]
            
            w_center = cw_curr * cw_prev
            w_left = cw_curr
            w_right = cw_prev
            
            denominator = w_center + w_left + w_right + 1e-6
            
            # 最終 Ratio [Batch-1] (因為 diff 會少一個維度)
            # 注意：這裡維度對齊非常痛苦，建議如果是初期復刻，
            # 強烈建議先用 Fixed Ratio (loss_ratio != 0)。
            # Dynamic Weight 很容易因為 Batch Size 太小而算出一堆 NaN。
            
            ratio_center = w_center / denominator
            ratio_left = w_left / denominator
            ratio_right = w_right / denominator
            
            # 計算 Loss
            # 因為使用了 padding 和 shifting，這裡的 batch size 會縮減
            # 我們需要對齊 loss 和 ratio 的長度
            
            # 取中間段的 loss
            l_c = self.base_loss(y_true[1:-1], y_pred[1:-1])
            l_l = self.base_loss(y_true[:-2], y_pred[1:-1]) # y_true[t-1] vs y_pred[t]
            l_r = self.base_loss(y_true[2:], y_pred[1:-1])  # y_true[t+1] vs y_pred[t]
            
            # 確保 ratio 長度跟 loss 一樣 (Batch - 2)
            # 這裡做一個簡單的 slice
            valid_len = l_c.shape[0]
            r_c = ratio_center[:valid_len].view(-1, 1) # view 用於廣播
            r_l = ratio_left[:valid_len].view(-1, 1)
            r_r = ratio_right[:valid_len].view(-1, 1)

            final_loss = torch.mean(r_c * l_c + r_l * l_l + r_r * l_r)
            return final_loss

        else:
            return self.base_loss(y_true, y_pred)

#### model

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass

"""
# 1. 配置管理 (取代 argparse)
@dataclass
class ModelConfig:
    block_channels: list = (64, 128, 256, 512) # 範例深度
    block_conv: int = 2        # 每個 Block 有幾層 Conv
    dropout_rate: float = 0.25
    n_frames: int = 3          # n=2 或 n=3 (對應 prev, curr, next)
    input_channels: int = 1    # 灰階為 1，RGB 為 3
    num_classes: int = 2       # 最終輸出的類別數 (對應 output_layer)
"""

# 2. 基礎卷積塊 (取代 set_or_get_conv2d)
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, num_layers, dropout_rate):
        super().__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            # 第一層負責改變通道數，後續層維持通道數
            cin = in_ch if i == 0 else out_ch
            self.layers.append(nn.Sequential(
                nn.Conv2d(cin, out_ch, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True)
            ))
        self.pool = nn.MaxPool2d(2)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        skip = x  # 保存用於 Skip Connection 的特徵 (Before Pooling)
        out = self.pool(x)
        out = self.dropout(out)
        return out, skip

# 3. 上採樣塊 (取代 Decoder 中的重複邏輯)
class UpBlock(nn.Module):
    def __init__(self, in_ch, skip_ch, out_ch, dropout_rate):
        super().__init__()
        # 對應 Conv2DTranspose(strides=2)
        self.up = nn.ConvTranspose2d(in_ch, out_ch, kernel_size=2, stride=2)
        self.bn_up = nn.BatchNorm2d(out_ch)
        
        # 融合後的卷積 (Skip connection channel size + Upsampled channel size)
        # 根據舊代碼：concatenate([curr_skip, prev_skip, up]) -> 2 * skip_ch + out_ch
        self.conv = nn.Sequential(
            nn.Conv2d(out_ch + 2 * skip_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x, skip_curr, skip_prev):
        x = self.up(x)
        x = self.bn_up(x)
        
        # 確保維度匹配 (處理 padding 問題)
        if x.size() != skip_curr.size():
            x = F.interpolate(x, size=skip_curr.shape[2:], mode='bilinear', align_corners=True)

        # Concatenate: [Curr, Prev, Up]
        cat_x = torch.cat([skip_curr, skip_prev, x], dim=1)
        return self.conv(cat_x)

from dataclasses import dataclass, field
from typing import List

@dataclass
class ModelConfig:
    # 1. 輸入與輸出規格
    input_channels: int = 1      # 輸入圖片的通道數 (醫學影像通常是 1，若是 RGB 則是 3)
    num_classes: int = 1         # 最終輸出的類別數 (若是做二元分割如：肝臟/背景，則為 1)
    
    # 2. 模型深度與寬度
    # 這決定了 U-Net 有幾層，以及每層變多厚
    # [64, 128, 256, 512] 代表有 4 個 Encoder Block，最底層是 512
    block_channels: List[int] = field(default_factory=lambda: [64, 128, 256, 512])
    
    # 3. 區塊內部的複雜度
    block_conv: int = 2          # 每個 ConvBlock 裡面重複做幾次卷積 (num_layers)
    
    # 4. 時序相關 (Concurrent 的核心)
    n_frames: int = 3            # 一次輸入幾張圖？(2 or 3) 這會影響 Bottleneck 的輸入厚度
    
    # 5. 正則化參數
    dropout_rate: float = 0.5    # Dropout 的比例

# 4. 主模型：Concurrent CRNN (Siamese U-Net)
class ConcurrentUNet(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.config = config
        
        # --- Encoder (權重共享) ---
        self.encoder_blocks = nn.ModuleList()
        in_ch = config.input_channels
        
        for out_ch in config.block_channels:
            self.encoder_blocks.append(
                ConvBlock(in_ch, out_ch, config.block_conv, config.dropout_rate)
            )
            in_ch = out_ch # 更新下一層的輸入
            
        # --- Bottleneck (最底層) ---
        # 根據舊代碼：如果是 n=3，concatenate(prev, curr, next)
        # 輸入通道數 = 最後一層 Encoder Channel * n_frames
        bottleneck_in_ch = config.block_channels[-1] * config.n_frames 
        bottleneck_out_ch = config.block_channels[-1]
        
        self.bottleneck_conv = nn.Sequential(
            nn.Conv2d(bottleneck_in_ch, bottleneck_out_ch, 3, padding=1),
            nn.BatchNorm2d(bottleneck_out_ch),
            nn.ReLU(inplace=True),
            nn.Dropout(config.dropout_rate)
        )

        # --- Decoder ---
        self.decoder_blocks = nn.ModuleList()
        # 倒序遍歷 channels (排除最後一層，因為那是 bottleneck 的輸入)
        rev_channels = config.block_channels[::-1] 
        
        for i in range(len(rev_channels) - 1):
            in_ch = rev_channels[i]
            skip_ch = rev_channels[i+1] # 下一層的 channel (因為是倒序)
            out_ch = rev_channels[i+1]
            
            self.decoder_blocks.append(
                UpBlock(in_ch, skip_ch, out_ch, config.dropout_rate)
            )
            
        # 最終輸出層
        self.final_conv = nn.Conv2d(config.block_channels[0], config.num_classes, kernel_size=1)

    def forward_single_branch(self, x):
        """跑單一分支的 Encoder"""
        skips = []
        for block in self.encoder_blocks:
            x, skip = block(x)
            skips.append(skip)
        return x, skips

    def forward(self, x_prev, x_curr, x_next=None):
        # 1. Encoder 階段 (權重共享：同一個 self.encoder_blocks 跑多次)
        feat_prev, skips_prev = self.forward_single_branch(x_prev)
        feat_curr, skips_curr = self.forward_single_branch(x_curr)
        
        feats = [feat_prev, feat_curr]
        if self.config.n_frames == 3 and x_next is not None:
            feat_next, _ = self.forward_single_branch(x_next)
            feats.append(feat_next) # [Prev, Curr, Next]
        
        # 2. Bottleneck 融合
        # 舊代碼邏輯：concatenate([prev_last, curr_last, (next_last)])
        bottleneck = torch.cat(feats, dim=1) 
        x = self.bottleneck_conv(bottleneck)
        
        # 3. Decoder 階段
        # 需要倒序取出 skips (因為 Decoder 是從底層往上)
        # skips 列表是 [Block0, Block1, Block2, Block3(Bottleneck input)]
        # 我們需要從 Block2 開始往回拿
        
        for i, up_block in enumerate(self.decoder_blocks):
            # 取得對應層級的 skip connection
            # i=0 時，處理的是倒數第二層的特徵
            idx = -(i + 2) 
            s_curr = skips_curr[idx]
            s_prev = skips_prev[idx]
            
            x = up_block(x, s_curr, s_prev)
            
        # 4. 輸出
        logits = self.final_conv(x)
        return logits

# --- 測試區 ---
if __name__ == '__main__':
    # 模擬輸入 (Batch=1, Channel=1, H=256, W=256)
    dummy_prev = torch.randn(1, 1, 256, 256)
    dummy_curr = torch.randn(1, 1, 256, 256)
    dummy_next = torch.randn(1, 1, 256, 256)

    # 初始化配置與模型
    cfg = ModelConfig(n_frames=3, block_channels=[32, 64, 128])
    model = ConcurrentUNet(cfg)
    
    # 執行 Forward Pass
    print("Model initialized. Running forward pass...")
    output = model(dummy_prev, dummy_curr, dummy_next)
    print("Output shape:", output.shape) # 預期: [1, 2, 256, 256]

Model initialized. Running forward pass...
Output shape: torch.Size([1, 1, 256, 256])


#### matrices

In [ ]:
import numpy as np
import pandas as pd

import torch
import numpy as np

def calculate_metrics(pred_logits, target, threshold=0.5):
    """
    一次計算所有指標：Dice, IoU (OM), Accuracy
    pred_logits: (B, 1, H, W) 未經過 Sigmoid 的輸出
    target: (B, 1, H, W) 0 或 1
    """
    # 1. 前處理
    pred_probs = torch.sigmoid(pred_logits)
    pred_bin = (pred_probs > threshold).float()
    
    # 為了計算方便，展平
    pred_flat = pred_bin.view(-1)
    target_flat = target.view(-1)
    
    # 2. 核心數值
    intersection = (pred_flat * target_flat).sum()
    pred_sum = pred_flat.sum()
    target_sum = target_flat.sum()
    
    # epsilon 防止除以 0
    eps = 1e-6
    
    # 3. 計算指標
    # Dice Score
    dice = (2. * intersection + eps) / (pred_sum + target_sum + eps)
    
    # IoU (Overlap Metric - OM)
    # Union = A + B - Inter
    union = pred_sum + target_sum - intersection
    iou = (intersection + eps) / (union + eps)
    
    # Accuracy (Pixel-wise)
    # 正確預測的像素數 / 總像素數
    correct = (pred_flat == target_flat).float().sum()
    total = torch.tensor(target_flat.numel()).float().to(pred_flat.device)
    acc = correct / total
    
    return {
        "dice": dice.item(),
        "om": iou.item(),  # 學長的 OM 就是 IoU
        "acc": acc.item()
    }

class VolumeMetrics:
    def __init__(self):
        self.reset()

    def reset(self):
        self.intersection = 0
        self.union = 0
        self.pred_sum = 0
        self.gt_sum = 0
        self.fp_count = 0
        self.tn_count = 0

    def update(self, pred, gt):
        """
        輸入單張或多張 Slice 的預測結果與 GT
        pred: (B, H, W) 0 or 1 (Binary)
        gt:   (B, H, W) 0 or 1 (Binary)
        """
        # 轉成 numpy 並確保是 boolean/int
        if hasattr(pred, 'cpu'): pred = pred.cpu().numpy()
        if hasattr(gt, 'cpu'): gt = gt.cpu().numpy()
        
        pred = pred.astype(np.uint8)
        gt = gt.astype(np.uint8)

        # 核心計算 (與學長邏輯一致)
        self.intersection += np.sum(pred * gt)
        self.pred_sum += np.sum(pred)
        self.gt_sum += np.sum(gt)
        
        # 用於計算 FP (學長定義: FP = (Pred總和 / GT總和) - Sensitivity)
        # 或者傳統定義: FP pixels = pred & (1-gt)
        self.fp_count += np.sum(pred * (1 - gt)) 
        
    def get_metrics(self):
        """回傳計算好的字典"""
        # 避免除以 0
        epsilon = 1e-6
        
        # 1. Overlap Metric (OM) / IoU
        # 學長公式: inter / (pred + gt - inter)
        om = self.intersection / (self.pred_sum + self.gt_sum - self.intersection + epsilon)
        
        # 2. True Positive Rate (TP) / Sensitivity
        # 學長公式: inter / gt
        tp = self.intersection / (self.gt_sum + epsilon)
        
        # 3. False Positive Rate (FP) - Ansen's definition
        # 學長公式: (pred_sum / gt_sum) - sensitivity
        # 數學上等價於: (pred_sum - intersection) / gt_sum = fp_pixels / gt_sum
        fp = self.fp_count / (self.gt_sum + epsilon)
        
        # 4. Dice Score (DR)
        # 學長公式: 2 * inter / (pred + gt)
        dr = 2 * self.intersection / (self.pred_sum + self.gt_sum + epsilon)

        return {
            "OM": om,
            "TP": tp,
            "FP": fp,
            "Dice": dr,
            "GT_Vol": self.gt_sum,
            "Pred_Vol": self.pred_sum
        }

#### inference

In [9]:
import os
import torch
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
from torch.utils.data import DataLoader



def save_prediction_image(pred, gt, img, save_path):
    """
    畫出 Input, GT, Pred 的對比圖 (參考學長的 plot_result)
    """
    # 轉回 0~255
    img = (img * 255).astype(np.uint8)
    gt = (gt * 255).astype(np.uint8)
    pred = (pred * 255).astype(np.uint8)
    
    # 簡單拼接: [Input, GT, Pred]
    # 你可以依照需求改成疊圖 (Overlay)
    concat = np.hstack([img, gt, pred])
    cv2.imwrite(save_path, concat)

def evaluate_all():
    cfg = Config()
    device = torch.device(cfg.DEVICE if torch.cuda.is_available() else "cpu")
    
    # 1. 載入模型
    print("Loading Model...")
    # 這裡你需要先跑一次 train.py 產生 best_model.pth
    model_path = os.path.join(cfg.CHECKPOINT_DIR, "best_model.pth")
    if not os.path.exists(model_path):
        print(f"Model not found at {model_path}! Please train first.")
        return

    # 計算 Input Channels (需與 Training 一致)
    # 這裡簡單假設你沒開 grad/blur，若是 config 有開要同步邏輯
    dummy_ds = CRNNDataset(cfg.TRAIN_IMG_DIR, n_frames=cfg.N_FRAMES, 
                           use_blur=cfg.USE_BLUR, use_grad=cfg.USE_GRAD)
    in_ch = dummy_ds[0]['curr'].shape[0]

    model_cfg = ModelConfig(
        block_channels=cfg.BLOCK_CHANNELS,
        n_frames=cfg.N_FRAMES,
        input_channels=in_ch,
        num_classes=cfg.NUM_CLASSES
    )
    model = ConcurrentUNet(model_cfg).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    # 2. 準備測試資料 (按 Case 處理)
    # 假設 TEST_IMG_DIR 下面是各個 Case 的資料夾
    test_root = './data/all/test'          # 請修改為實際路徑
    test_target = './data/all/test_target' # 請修改為實際路徑
    output_dir = './evaluation_results'
    os.makedirs(output_dir, exist_ok=True)
    
    # 取得所有 Case 名稱
    case_names = sorted([d for d in os.listdir(test_root) 
                        if os.path.isdir(os.path.join(test_root, d))])
    
    all_results = []
    
    print(f"Found {len(case_names)} cases to evaluate.")

    # 3. 逐一 Case 評估
    for case_name in tqdm(case_names, desc="Evaluating Cases"):
        case_img_dir = os.path.join(test_root, case_name)
        case_target_dir = os.path.join(test_target, case_name) # 假設結構對稱
        
        # 針對單一 Case 建立 Dataset
        # 注意：這裡 root_dir 直接指到 case 資料夾，所以 Dataset 內部的邏輯要能處理
        # 我們之前寫的 Dataset 預設 root_dir 下面是 case folders
        # 為了方便，我們這裡還是指向 root，但在 Dataset 內部我們只會讀取特定 case
        # 或者：簡單一點，我們直接修改 Dataset 讓它接受 list of files
        # 這裡採用最簡單方法：直接用我們寫好的 Dataset，但 root 指向 case 的上一層，
        # 然後用 dataset 內部的 filter? 不，這樣太慢。
        
        # --- 權宜之計：直接在這裡實例化 Dataset 讀取單一資料夾 ---
        # 為了不改動 Dataset.py 太大，建議 Dataset 支援 "Single Case Mode"
        # 只要傳入 case_img_dir 當作 root，且 Dataset 發現下面沒有子資料夾時，就會把它當作單一 case 處理
        # (我之前給你的 Dataset 已經包含了這個邏輯 check: if not case_dirs...)
        
        ds = CRNNDataset(
            root_dir=case_img_dir, 
            target_dir=case_target_dir,
            n_frames=cfg.N_FRAMES,
            use_blur=cfg.USE_BLUR,
            use_grad=cfg.USE_GRAD
        )
        
        loader = DataLoader(ds, batch_size=cfg.BATCH_SIZE, shuffle=False) # 絕對不能 Shuffle
        
        metrics = VolumeMetrics()
        
        # 建立保存圖片的資料夾
        save_img_dir = os.path.join(output_dir, "plots", case_name)
        os.makedirs(save_img_dir, exist_ok=True)

        with torch.no_grad():
            for i, batch in enumerate(loader):
                x_prev = batch['prev'].to(device)
                x_curr = batch['curr'].to(device)
                x_next = batch['next'].to(device) if 'next' in batch else None
                y_true = batch['label'].to(device)

                # Inference
                logits = model(x_prev, x_curr, x_next)
                preds = torch.sigmoid(logits)
                preds_bin = (preds > 0.5).float() # 二值化

                # 更新指標
                metrics.update(preds_bin, y_true)

                # 保存圖片 (只存每個 Batch 的第一張示意，避免存太多)
                # 這裡需要小心 tensor 轉 numpy 的維度
                # batch['curr'] 是 (B, C, H, W)，我們只取 Channel 0 (原圖)
                input_img = batch['curr'][0, 0].cpu().numpy() 
                gt_img = y_true[0, 0].cpu().numpy()
                pred_img = preds_bin[0, 0].cpu().numpy()
                
                # 存檔名包含 Batch Index
                save_path = os.path.join(save_img_dir, f"slice_{i*cfg.BATCH_SIZE}.png")
                save_prediction_image(pred_img, gt_img, input_img, save_path)

        # 計算該 Case 的最終分數
        res = metrics.get_metrics()
        res['Case'] = case_name
        all_results.append(res)
        
        # 顯示當前 Case 的 OM
        # tqdm.write(f"Case {case_name}: OM={res['OM']:.4f}")

    # 4. 輸出報表
    df = pd.DataFrame(all_results)
    # 把 Case 移到第一欄
    cols = ['Case'] + [c for c in df.columns if c != 'Case']
    df = df[cols]
    
    csv_path = os.path.join(output_dir, "case_result.csv")
    df.to_csv(csv_path, index=False)
    print(f"\nEvaluation Complete! Results saved to {csv_path}")
    print(f"Average OM: {df['OM'].mean():.4f}")

if __name__ == '__main__':
    evaluate_all()

NameError: name 'Config' is not defined

#### train

In [ ]:
import os
import time
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import json

# 匯入專案模組
from config import Config
from dataset import CRNNDataset
from model import ConcurrentUNet, ModelConfig
from loss import WeightedConsecutiveLoss
from utils import calculate_metrics

def train():
    # --- 1. 初始化與配置 ---
    cfg = Config()
    
    # 建立存檔目錄
    save_dir = cfg.CHECKPOINT_DIR
    os.makedirs(save_dir, exist_ok=True)
    
    # 記錄檔 (對應學長的 logs.json)
    log_file = os.path.join(save_dir, 'training_log.json')
    
    device = torch.device(cfg.DEVICE if torch.cuda.is_available() else "cpu")
    print(f"Start Training on {device}...")

    # --- 2. 資料準備 ---
    # 這裡我們將資料集切分為 Train (80%) 和 Validation (20%)
    # 學長是用 split_num 手動切，這裡用 PyTorch 標準做法
    full_ds = CRNNDataset(
        root_dir=cfg.TRAIN_IMG_DIR,
        target_dir=cfg.TRAIN_MASK_DIR,
        n_frames=cfg.N_FRAMES,
        use_blur=cfg.USE_BLUR,
        use_grad=cfg.USE_GRAD
    )
    
    # 自動偵測 channel 數
    sample_ch = full_ds[0]['curr'].shape[0]
    print(f"Detected Input Channels: {sample_ch}")
    
    val_size = int(len(full_ds) * 0.2)
    train_size = len(full_ds) - val_size
    train_ds, val_ds = random_split(full_ds, [train_size, val_size])
    
    print(f"Data Split: Train={len(train_ds)}, Val={len(val_ds)}")

    train_loader = DataLoader(train_ds, batch_size=cfg.BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=cfg.BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    # --- 3. 模型與優化器 ---
    model_cfg = ModelConfig(
        block_channels=cfg.BLOCK_CHANNELS,
        dropout_rate=cfg.DROPOUT_RATE,
        n_frames=cfg.N_FRAMES,
        input_channels=sample_ch,
        num_classes=cfg.NUM_CLASSES
    )
    model = ConcurrentUNet(model_cfg).to(device)

    criterion = WeightedConsecutiveLoss(loss_ratio=0.5, consecutive=cfg.N_FRAMES).to(device)
    optimizer = optim.Adam(model.parameters(), lr=cfg.LEARNING_RATE)
    scaler = torch.cuda.amp.GradScaler()

    # --- 4. 監控變數 (還原學長的 Checkpoint 邏輯) ---
    best_metrics = {
        'val_loss': float('inf'), # Min
        'val_om': float('-inf'),  # Max
        'train_loss': float('inf'),
        'train_om': float('-inf')
    }
    
    # --- 5. 訓練迴圈 ---
    for epoch in range(cfg.EPOCHS):
        start_time = time.time()
        
        # === Training Phase ===
        model.train()
        train_log = {'loss': 0, 'dice': 0, 'om': 0, 'acc': 0}
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{cfg.EPOCHS} [Train]")
        for batch in pbar:
            # 搬運資料
            x_prev = batch['prev'].to(device)
            x_curr = batch['curr'].to(device)
            x_next = batch['next'].to(device) if 'next' in batch else None
            y_true = batch['label'].to(device)

            optimizer.zero_grad()
            
            # Forward (Mixed Precision)
            with torch.cuda.amp.autocast():
                logits = model(x_prev, x_curr, x_next)
                loss = criterion(y_true, torch.sigmoid(logits)) # Loss 內部通常預期 0~1 的 input
            
            # Backward
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            # 計算指標
            metrics = calculate_metrics(logits, y_true)
            
            # 更新 Log
            train_log['loss'] += loss.item()
            train_log['dice'] += metrics['dice']
            train_log['om'] += metrics['om']
            train_log['acc'] += metrics['acc']
            
            pbar.set_postfix({'loss': loss.item(), 'om': metrics['om']})

        # 平均化 Train Metrics
        for k in train_log: train_log[k] /= len(train_loader)

        # === Validation Phase ===
        model.eval()
        val_log = {'loss': 0, 'dice': 0, 'om': 0, 'acc': 0}
        
        with torch.no_grad():
            for batch in val_loader:
                x_prev = batch['prev'].to(device)
                x_curr = batch['curr'].to(device)
                x_next = batch['next'].to(device) if 'next' in batch else None
                y_true = batch['label'].to(device)

                logits = model(x_prev, x_curr, x_next)
                loss = criterion(y_true, torch.sigmoid(logits))
                
                metrics = calculate_metrics(logits, y_true)
                
                val_log['loss'] += loss.item()
                val_log['dice'] += metrics['dice']
                val_log['om'] += metrics['om']
                val_log['acc'] += metrics['acc']
        
        # 平均化 Val Metrics
        for k in val_log: val_log[k] /= len(val_loader)

        # === 結算與存檔 (還原學長的 4 個 Checkpoint) ===
        epoch_time = time.time() - start_time
        print(f"\nSummary Ep {epoch+1} | Time: {epoch_time:.1f}s")
        print(f"Train | Loss: {train_log['loss']:.4f} | OM: {train_log['om']:.4f} | Dice: {train_log['dice']:.4f}")
        print(f"Val   | Loss: {val_log['loss']:.4f} | OM: {val_log['om']:.4f} | Dice: {val_log['dice']:.4f}")

        # 1. Save Best Val Loss (model_loss.h5)
        if val_log['loss'] < best_metrics['val_loss']:
            best_metrics['val_loss'] = val_log['loss']
            torch.save(model.state_dict(), os.path.join(save_dir, 'best_val_loss.pth'))
            print("✅ Saved Best Val Loss Model")

        # 2. Save Best Val OM (model_om.h5)
        if val_log['om'] > best_metrics['val_om']:
            best_metrics['val_om'] = val_log['om']
            torch.save(model.state_dict(), os.path.join(save_dir, 'best_val_om.pth'))
            print("✅ Saved Best Val OM Model")

        # 3. Save Best Train Loss (model_train_loss.h5) - 選用
        if train_log['loss'] < best_metrics['train_loss']:
            best_metrics['train_loss'] = train_log['loss']
            # torch.save(model.state_dict(), os.path.join(save_dir, 'best_train_loss.pth'))

        # 4. Save Best Train OM (model_train_om.h5) - 選用
        if train_log['om'] > best_metrics['train_om']:
            best_metrics['train_om'] = train_log['om']
            # torch.save(model.state_dict(), os.path.join(save_dir, 'best_train_om.pth'))

        # 寫入 JSON Log
        log_entry = {
            'epoch': epoch + 1,
            'time': epoch_time,
            'train': train_log,
            'val': val_log
        }
        with open(log_file, 'a') as f:
            f.write(json.dumps(log_entry) + "\n")

if __name__ == '__main__':
    train()